In [1]:
import pandas as pd
import unidecode

# !pip install fuzzywuzzy
# !pip install python-Levenshtein

from fuzzywuzzy import fuzz


c:\Users\carlo\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [50]:
df_donors = pd.read_csv('past_donors_clean.csv', parse_dates=[1])
df_orgs = pd.read_excel('Organizations.xlsx')

In [34]:
#company names:princip filiales, name, courriel

def stdr_names(series_original):
    series = series_original.copy()
    series = series.astype(str)
    
    series = series.str.upper()
    series = series.str.strip()

    series = series.replace(r'\s+', ' ', regex=True)
    series = series.str.replace(r'[^\w\s]+', '', regex=True)


    series = series.apply(lambda x: unidecode.unidecode(x))

    series = series.str.replace(' QUEBEC ', '')
    series = series.str.replace('CANADA', '')
    series = series.str.replace(' MONTREAL ', '')
    series = series.str.replace('MONTREAL ', '')
    series = series.str.replace(' MONTREAL', '')
    series = series.str.replace(' INC', '')
    series = series.str.replace(' INC ', '')
    series = series.str.replace(' CIE ', '')
    series = series.str.replace(' CIE', '')
    series = series.str.replace(' LTEE', '')
    series = series.str.replace('CORPORATION', '')
    series = series.str.replace('INTERNATIONAL', '')


    return series


In [52]:
df_donors = df_donors[ df_donors.company.notna() ]

In [53]:
df_orgs.NAME = stdr_names(df_orgs.NAME)

df_donors.company = df_donors.company.astype(str)
df_donors.company  = stdr_names(df_donors.company)

In [45]:
pd.merge(df_orgs.NAME.drop_duplicates(), df_donors.company.drop_duplicates(), left_on='NAME', right_on='company')

# 18 matches by exact matching

,NAME,company
0,ARCHAMBAULT,ARCHAMBAULT
1,AVERNA,AVERNA
2,CANADIAN TIRE,CANADIAN TIRE
3,CENTRES DENTAIRES LAPOINTE,CENTRES DENTAIRES LAPOINTE
4,CGI,CGI
5,CORBEC,CORBEC
6,DORFIN,DORFIN
7,ENERGIE CARDIO,ENERGIE CARDIO
8,FIERA CAPITAL,FIERA CAPITAL
9,FUTURE SHOP,FUTURE SHOP


In [46]:
# https://www.datacamp.com/tutorial/fuzzy-string-python

def match_comp_names(orgs, donors):

    # orgs = orgs.NAME.drop_duplicates().to_list()
    # donors = donors.company.drop_duplicates().to_list()

    d = 0
    j = 0
    rows = []

    for org in orgs:
        for donor in donors:
            Str1 = org
            Str2 = donor
            Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
            Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
            Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
            Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)

            row = [org, donor, Ratio, Partial_Ratio, Token_Sort_Ratio, Token_Set_Ratio]

            rows.append(row)
    
    return rows

In [48]:
rows = match_comp_names(df_orgs.NAME.drop_duplicates().to_list(), df_donors.company.drop_duplicates().to_list())

df_sim = pd.DataFrame(rows, columns = ['org', 'donor', 'Ratio', 'Partial_Ratio', 'Token_Sort_Ratio', 'Token_Set_Ratio'] )

df_sim.to_pickle('similarity_score.pkl')


KeyboardInterrupt: 

In [142]:
score = 'Ratio'

df_ratios = df_sim.groupby('org', as_index=False)[score].max()
df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(30)

,org,Ratio,donor
1309,KRUGER,100,KRUGER
578,AVERNA,100,AVERNA
1461,MAXI,100,MAXI
1218,IKEA,100,IKEA
974,ENERGIE CARDIO,100,ENERGIE CARDIO
612,BDO,100,BDO
843,CORBEC,100,CORBEC
731,CANADIAN TIRE,100,CANADIAN TIRE
1272,JEAN COUTU,100,JEAN COUTU
1060,FUTURE SHOP,100,FUTURE SHOP


In [143]:
score = 'Token_Set_Ratio'

df_ratios = df_sim.groupby('donor', as_index=False)[score].max()
df_token_set_ratio = df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False)

In [144]:
df_token_set_ratio[df_token_set_ratio.Token_Set_Ratio.ge(85)].to_clipboard()

In [145]:
score = 'Partial_Ratio'

df_ratios = df_sim.groupby('org', as_index=False)[score].max()
df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(20)

,org,Partial_Ratio,donor
1807,DORFIN,100,DORFIN
4280,VA,100,BOULEVARD MUSIQUE
4291,VA,100,SALVAIL ET CO
4290,VA,100,EMILIE VANDAL
4289,VA,100,SYLVAIN ROCHON DEPUTE DE RICHELIEU
4288,VA,100,FESTIVAL WESTERN DE ST TITE
4287,VA,100,METRO TRACY ALIMENTATION SYLVAIN BRIERE
4286,VA,100,CENTRE GERVAIS
4285,VA,100,AVANTI CINE VIDEO
4284,VA,100,E VAGUE


In [146]:


score = 'Token_Sort_Ratio'

df_ratios = df_sim.groupby('org', as_index=False)[score].max()
df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(20)

,org,Token_Sort_Ratio,donor
756,CENTRES DENTAIRES LAPOINTE,100,CENTRES DENTAIRES LAPOINTE
1820,SNC LAVALIN,100,SNC LAVALIN
1762,ROYAL LEPAGE,100,ROYAL LEPAGE
1272,JEAN COUTU,100,JEAN COUTU
1987,UPS,100,UPS
925,DORFIN,100,DORFIN
839,CORBEC,100,CORBEC
1477,MERCER,100,MERCER
1626,PFIZER,100,PFIZER
1451,MAXI,100,MAXI


In [64]:
th = 70

df_filt = df_sim.query(f''' Ratio > {th} |  Token_Sort_Ratio > {th}  | Token_Set_Ratio > {th}  ''' ).reset_index(drop=True)


In [65]:
df_filt.shape

(294, 6)

In [53]:
df_filt.sort_values(['Token_Set_Ratio', 'Ratio'], ascending=False).to_clipboard(index=None)

### Merge Donors and Foundations

In [31]:
import pandas as pd

df_fonds = pd.read_excel('Foundations.xlsx')
df_orgs = pd.read_excel('Organizations.xlsx')

In [32]:
df_fonds.rename(columns= {'NAME':'FOND_NAME'}, inplace=True)

In [33]:
df_fonds = df_fonds.merge(df_orgs[['ID', 'NAME']], on = 'ID').rename(columns={'NAME':'COMP_NAME'})

In [37]:
df_fonds = df_fonds[ df_fonds.FOND_NAME.notna() ]

In [39]:
df_fonds[['COMP_NAME', 'FOND_NAME']]

,COMP_NAME,FOND_NAME
0,AbbVie,"AbbVie,"
1,Air Canada,"Air-Canada,"
2,Alcoa Canada Première fusion,Alcoa Foundation
3,Alfred Dallaire / Memoria,Fondation Memoria
4,Allstate du Canada,Allstate Foundation of Canada
...,...,...
75,Ultramar Ltée,"Utramar Canada,"
77,UPS Canada,"UPS Foundation,"
78,Velan inc.,"Velan Foundation,"
79,Vêtements Peerless,"Segal Family Foundation,"


In [54]:
df_fonds.FOND_NAME = stdr_names(df_fonds.FOND_NAME)

rows = match_comp_names(df_fonds.FOND_NAME.drop_duplicates().to_list(), df_donors.company.drop_duplicates().to_list())

df_sim = pd.DataFrame(rows, columns = ['fondation', 'donor', 'Ratio', 'Partial_Ratio', 'Token_Sort_Ratio', 'Token_Set_Ratio'] )


In [56]:
th = 70

df_filt = df_sim.query(f''' Ratio > {th} |  Token_Sort_Ratio > {th}  | Token_Set_Ratio > {th}  ''' ).reset_index(drop=True)


In [59]:
df_filt.sort_values(['Token_Set_Ratio', 'Ratio'], ascending=False).to_clipboard(index=None)

In [ ]:
df_filt

In [ ]:
## continue: add fondation columns to the matched rows

In [2]:
import pandas as pd
# !pip install fuzzywuzzy
# !pip install python-Levenshtein

In [130]:
df_donors = pd.read_csv('past_donors_clean.csv', parse_dates=[1])
df_orgs = pd.read_excel('Organizations.xlsx')
df_matches = pd.read_excel('matched_orgs_k4kreview.xlsx')

df_donors.company

df_donors.company = stdr_names(df_donors.company)
df_orgs.NAME = stdr_names(df_orgs.NAME)

In [131]:
df_matches = df_matches[df_matches.Match == 1]
df_matches = df_matches.iloc[:, 0:2]
df_matches.columns = ['ORGANIZATION', 'DONOR']

In [132]:
donor_q = df_matches.DONOR.value_counts()
organization_q = df_matches.ORGANIZATION.value_counts()

In [133]:
df1 = df_matches[df_matches.ORGANIZATION.isin(organization_q[organization_q > 1].index)].sort_values('ORGANIZATION').reset_index(drop=True)

df1

,ORGANIZATION,DONOR
0,BEAULIEU,DANIEL BEAULIEU
1,BEAULIEU,GROUPE C G BEAULIEU
2,BUROPRO CITATION,BUROPRO CITATION BELOEIL
3,BUROPRO CITATION,LIBRAIRIE BUROPRO CITATION
4,DESJARDINS CAISSE CENTRALE,CAISSE DESJARDINS SAINT MATHIAS
5,DESJARDINS CAISSE CENTRALE,CAISSE DESJARDINS RICHELIEU SAINT MATHIAS
6,DESJARDINS FIDUCIE,CAISSE DESJARDINS SAINT MATHIAS
7,DESJARDINS FIDUCIE,CAISSE DESJARDINS RICHELIEU SAINT MATHIAS
8,FINANCIERE SUN LIFE DU,SUNLIFE FINANCIERE
9,FINANCIERE SUN LIFE DU,SUN LIFE FINANCIAL


In [134]:
df1 = df_matches[df_matches.DONOR.isin(donor_q[donor_q > 1].index)].sort_values('DONOR').reset_index(drop=True)

In [135]:
ixs = [1, 2,5,6,10,11,15,16,18, 21,22]

df1 = df1.iloc[ixs,:].reset_index(drop=True)

In [136]:
df_matches = pd.concat([ df_matches[df_matches.DONOR.isin(donor_q[donor_q <= 1].index)].sort_values('DONOR').reset_index(drop=True),
            df1 ] )

In [137]:
df_matches

,ORGANIZATION,DONOR
0,ALOUETTES DE,ALOUETTES
1,ARCHAMBAULT,ARCHAMBAULT
2,AVERNA,AVERNA
3,AVIS LOCATION DE VOITURES,AVIS
4,BMR,BMR OSTIGUY FRERES
...,...,...
6,RBC GROUPE FINANCIER,RBC
7,SNCLAVALIN,SNC LAVALIN
8,FINANCIERE SUN LIFE DU,SUNLIFE FINANCIERE
9,UPS,UPS


In [138]:
df_donors_match = df_donors.merge(df_matches, how = 'inner', left_on = 'company', right_on = 'DONOR')

In [139]:
df_donors_match.shape

(177, 8)

In [140]:
df_donors_match = df_donors_match.merge(df_orgs, how = 'inner', left_on = 'ORGANIZATION', right_on = 'NAME')

In [141]:
df_donors_match.DONOR.value_counts()

DELTA                       16
WESTJET                     13
CN                          12
BDO                          9
DELMAR                       6
                            ..
DANIEL BEAULIEU              1
BUROPRO CITATION BELOEIL     1
KSH                          1
DORFIN                       1
AVERNA                       1
Name: DONOR, Length: 61, dtype: int64

In [144]:
df_donors_match = df_donors_match.drop_duplicates().reset_index(drop=True)

In [ ]:
# add foundations based on organization id

In [145]:
df_donors_match

,company,added_modified,dollar_amount_float,moneyprize_received,donation_details,year,ORGANIZATION,DONOR,ID,2E CONTACT POUR,...,LIMITES GEOG,N DE TEL,NAME,NBRE DE SUCC,NOMBRE DEMPLOYES,NOTE,POSTE,PRINCIP FILIALES,SECTEUR INDUSTRIEL,SITE WEB
0,VIA,2010-01-19,0.0,NaN,trip for 4 Montreal/Windsor $2600 value,2010,VIA RAIL,VIA,1312,NaN,...,nationales.,514-871-6000; 514-871-6000; 800-799-9934,VIA RAIL,NaN,1 352,Toute demande de dons ou de commandites doit ê...,NaN,NaN,Transport; Services ferroviaires pour passagers.,www.viarail.ca
1,VIA,2011-01-01,0.0,Yes,trip for 4 Montreal/Windsor $2600 value,2011,VIA RAIL,VIA,1312,NaN,...,nationales.,514-871-6000; 514-871-6000; 800-799-9934,VIA RAIL,NaN,1 352,Toute demande de dons ou de commandites doit ê...,NaN,NaN,Transport; Services ferroviaires pour passagers.,www.viarail.ca
2,VIA,2012-01-09,0.0,Yes,trip for 4 Montreal/Windsor $2251 value,2012,VIA RAIL,VIA,1312,NaN,...,nationales.,514-871-6000; 514-871-6000; 800-799-9934,VIA RAIL,NaN,1 352,Toute demande de dons ou de commandites doit ê...,NaN,NaN,Transport; Services ferroviaires pour passagers.,www.viarail.ca
3,VIA,2012-06-12,0.0,Yes,trip for 4 Montreal/Windsor $2251 value,2013,VIA RAIL,VIA,1312,NaN,...,nationales.,514-871-6000; 514-871-6000; 800-799-9934,VIA RAIL,NaN,1 352,Toute demande de dons ou de commandites doit ê...,NaN,NaN,Transport; Services ferroviaires pour passagers.,www.viarail.ca
4,VIA,2014-03-04,0.0,Yes,Silver,2014,VIA RAIL,VIA,1312,NaN,...,nationales.,514-871-6000; 514-871-6000; 800-799-9934,VIA RAIL,NaN,1 352,Toute demande de dons ou de commandites doit ê...,NaN,NaN,Transport; Services ferroviaires pour passagers.,www.viarail.ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,JEAN COUTU,2018-03-07,0.0,Yes,2 gift certificates,2018,JEAN COUTU,JEAN COUTU,677,"CommanditeMadame Raphaële Charette, Directeure...",...,régionales.,450-646-9760; 450-646-9760,JEAN COUTU,NaN,18 557,Sur le site web sélectionner : Engagement com...,NaN,Centre d'information Rx,Commerce de gros et détail; Chaîne de pharmacies.,www.jeancoutu.com
173,CENTRES DENTAIRES LAPOINTE,2018-03-09,500.0,No,$500 donation,2018,CENTRES DENTAIRES LAPOINTE,CENTRES DENTAIRES LAPOINTE,259,NaN,...,régionales.,514-527-6468; 450-679-2300; 800-527-6468,CENTRES DENTAIRES LAPOINTE,NaN,376,NaN,NaN,NaN,Produits et services de soins personnels; Dent...,www.centreslapointe.com/
174,CIBC,2018-03-22,200.0,Yes,$200 1/4 page ad,2018,BANQUE CIBC,CIBC,121,NaN,...,nationales.,800-465-2422; 416-980-2211,BANQUE CIBC,NaN,2 841,Sur le site web sélectionner : À propos de la ...,NaN,"Gestion d'actifs CIBC, Gestion globale d'actif...","Services financiers; Finances et assurances, s...",www.cibc.com
175,UPS,2018-04-02,1000.0,Yes,$1000 donation,2018,UPS,UPS,1286,NaN,...,nationales.,800-742-5877; 905-676-1708,UPS,NaN,1 800,Toute demande de dons doit être envoyée par la...,NaN,"UPS Logistics, UPS Ventures, UPS Customhouse, ...",Transport; Services personnalisés: messagerie ...,www.ups.com
